In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os

from sklearn.metrics import classification_report, confusion_matrix


In [2]:
train_dir = "data/train"
val_dir = "data/val"
test_dir = "data/test"

img_size = (224, 224)
batch_size = 32
num_classes = 5  # tf_flowers has 5 classes


In [3]:
train_gen = ImageDataGenerator(
    rescale=1/255.0,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

val_gen = ImageDataGenerator(rescale=1/255.0)
test_gen = ImageDataGenerator(rescale=1/255.0)

train_data = train_gen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical"
)

val_data = val_gen.flow_from_directory(
    val_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical"
)

test_data = test_gen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False
)

class_names = list(train_data.class_indices.keys())
print("Classes:", class_names)


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'data/train'

In [ ]:
base_model = ResNet50(
    weights="imagenet",
    include_top=False,
    input_shape=(224, 224, 3)
)

# Phase 1: Freeze base model
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation="relu")(x)
x = Dropout(0.3)(x)
output = Dense(num_classes, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()


In [ ]:
callbacks = [
    ModelCheckpoint("best_model.h5", save_best_only=True, monitor="val_accuracy"),
    EarlyStopping(monitor="val_accuracy", patience=5, restore_best_weights=True)
]

history1 = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10,
    callbacks=callbacks
)


In [ ]:
# Unfreeze the top 50 layers of the base model
for layer in base_model.layers[-50:]:
    layer.trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),  # Very low LR
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

history2 = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10,
    callbacks=callbacks
)


In [ ]:
def plot_history(history, title):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    plt.figure(figsize=(12,5))
    plt.suptitle(title)

    plt.subplot(1,2,1)
    plt.plot(acc, label="train accuracy")
    plt.plot(val_acc, label="val accuracy")
    plt.legend()

    plt.subplot(1,2,2)
    plt.plot(loss, label="train loss")
    plt.plot(val_loss, label="val loss")
    plt.legend()

    plt.show()

plot_history(history1, "Phase 1 Training")
plot_history(history2, "Phase 2 Fine-tuning")


In [ ]:
model.load_weights("best_model.h5")

pred_probs = model.predict(test_data)
y_pred = np.argmax(pred_probs, axis=1)
y_true = test_data.classes

print(classification_report(y_true, y_pred, target_names=class_names))


In [ ]:
cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(8,6))
sns.heatmap(
    cm,
    annot=True,
    fmt="d",
    xticklabels=class_names,
    yticklabels=class_names,
    cmap="Blues"
)
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()


In [ ]:
import cv2

def get_gradcam(model, img_array, layer_name):
    grad_model = tf.keras.models.Model(
        [model.inputs], 
        [model.get_layer(layer_name).output, model.output]
    )
    
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        pred_index = tf.argmax(predictions[0])
        loss = predictions[:, pred_index]

    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    conv_outputs = conv_outputs[0]
    heatmap = tf.zeros(conv_outputs.shape[0:2])

    for i in range(conv_outputs.shape[-1]):
        heatmap += pooled_grads[i] * conv_outputs[:, :, i]

    heatmap = np.maximum(heatmap, 0)
    heatmap /= tf.math.reduce_max(heatmap)

    return heatmap.numpy()


In [ ]:
import matplotlib.image as mpimg

# Choose any test image path manually
img_path = r"D:\transfer-learning-image-classifier\data\test\daisy\daisy_3.jpg"

img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0) / 255.0

layer_name = "conv5_block3_out"  # Last convolutional block of ResNet50

heatmap = get_gradcam(model, img_array, layer_name)

plt.imshow(img)
plt.title("Original Image")
plt.show()


# Resize heatmap
heatmap = cv2.resize(heatmap, (224, 224))
heatmap = np.uint8(255 * heatmap)

# Apply color map
heatmap_color = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

# Convert original image to uint8
img_uint8 = (img_array[0] * 255).astype("uint8")

# Overlay heatmap on image
overlay = cv2.addWeighted(heatmap_color, 0.4, img_uint8, 0.6, 0)

plt.imshow(overlay)
plt.title("Grad-CAM")
plt.axis("off")
plt.show()


In [ ]:
baseline = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)),
    tf.keras.layers.MaxPooling2D(),
    
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax'),
])

baseline.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

baseline.summary()


In [ ]:
history_base = baseline.fit(
    train_data,
    validation_data=val_data,
    epochs=5
)


In [ ]:
baseline_pred = baseline.predict(test_data)
baseline_y_pred = np.argmax(baseline_pred, axis=1)

print("Baseline Model Performance:\n")
print(classification_report(y_true, baseline_y_pred, target_names=class_names))

In [ ]:
model.save("models/final_model.keras")


In [ ]:
best_model = tf.keras.models.load_model("models/best_model.h5")
best_model.save("models/final_model.keras")


In [ ]:
import tensorflow as tf

best_model = tf.keras.models.load_model("models/best_model.h5")
best_model.save("models/final_model.keras")


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8,5))
plt.plot(history.history["accuracy"], label="Train Acc")
plt.plot(history.history["val_accuracy"], label="Val Acc")
plt.legend()
plt.title("Accuracy Curve")
plt.savefig("visualizations/accuracy_curve.png")
plt.close()

plt.figure(figsize=(8,5))
plt.plot(history.history["loss"], label="Train Loss")
plt.plot(history.history["val_loss"], label="Val Loss")
plt.legend()
plt.title("Loss Curve")
plt.savefig("visualizations/loss_curve.png")
plt.close()


In [4]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8,5))
plt.plot(history.history["accuracy"], label="Train Acc")
plt.plot(history.history["val_accuracy"], label="Val Acc")
plt.legend()
plt.title("Accuracy Curve")
plt.savefig("visualizations/accuracy_curve.png")
plt.close()

plt.figure(figsize=(8,5))
plt.plot(history.history["loss"], label="Train Loss")
plt.plot(history.history["val_loss"], label="Val Loss")
plt.legend()
plt.title("Loss Curve")
plt.savefig("visualizations/loss_curve.png")
plt.close()


NameError: name 'history' is not defined

<Figure size 800x500 with 0 Axes>

In [5]:
import os

os.makedirs("visualizations", exist_ok=True)
print("Visualization folder ready.")


Visualization folder ready.


In [6]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=class_names, yticklabels=class_names)
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")

plt.savefig("visualizations/confusion_matrix.png")
plt.close()

print("Saved: visualizations/confusion_matrix.png")


NameError: name 'y_true' is not defined

In [7]:
test_data = tf.keras.preprocessing.image_dataset_from_directory(
    "data/test",
    image_size=(224,224),
    shuffle=False
)


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'data/test'

In [8]:
import os
os.getcwd()


'D:\\transfer-learning-image-classifier\\notebooks'

In [9]:
test_data = tf.keras.preprocessing.image_dataset_from_directory(
    "../data/test",
    image_size=(224,224),
    shuffle=False
)


Found 556 files belonging to 5 classes.


In [10]:
class_names = test_data.class_names
class_names


['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']

In [11]:
model = tf.keras.models.load_model("../models/best_model.h5")


In [12]:
import numpy as np

y_true = []
for _, labels in test_data:
    y_true.extend(labels.numpy())
y_true = np.array(y_true)

pred = model.predict(test_data)
y_pred = np.argmax(pred, axis=1)

print("Shapes:", y_true.shape, y_pred.shape)


18/18 ━━━━━━━━━━━━━━━━━━━━ 113s 6s/step
Shapes: (556,) (556,)


In [13]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(8,6))
sns.heatmap(
    cm,
    annot=True,
    fmt="d",
    cmap="Blues",
    xticklabels=class_names,
    yticklabels=class_names,
)

plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")

plt.savefig("../visualizations/confusion_matrix.png")
plt.close()

print("Confusion matrix saved successfully!")


Confusion matrix saved successfully!


In [14]:
../data/test/daisy/image_003.jpg

SyntaxError: invalid syntax (3798429895.py, line 1)

In [15]:
img_path = "../data/test/daisy/daisy_3.jpg"


In [16]:
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt

def get_gradcam(model, img_array, layer_name):
    grad_model = tf.keras.models.Model(
        inputs=model.inputs,
        outputs=[model.get_layer(layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        pred_index = tf.argmax(predictions[0])
        pred_score = predictions[:, pred_index]

    grads = tape.gradient(pred_score, conv_outputs)[0]

    pooled_grads = tf.reduce_mean(grads, axis=(0, 1))
    conv_outputs = conv_outputs[0]

    heatmap = tf.reduce_sum(tf.multiply(pooled_grads, conv_outputs), axis=-1)
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)

    return heatmap.numpy()


In [17]:
# Load an image
img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0) / 255.0

# Generate Grad-CAM
heatmap = get_gradcam(model, img_array, layer_name="conv5_block3_out")  # ResNet50 last conv layer


D:\python\Lib\site-packages\keras\src\models\functional.py:241: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['input_layer']
Received: inputs=Tensor(shape=(1, 224, 224, 3))
  warnings.warn(msg)


AttributeError: 'numpy.ndarray' object has no attribute 'numpy'

In [18]:
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt

def get_gradcam(model, img_array, layer_name):
    grad_model = tf.keras.models.Model(
        inputs=model.inputs,
        outputs=[model.get_layer(layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        pred_index = tf.argmax(predictions[0])
        pred_score = predictions[:, pred_index]

    grads = tape.gradient(pred_score, conv_outputs)[0]

    pooled_grads = tf.reduce_mean(grads, axis=(0, 1))
    conv_outputs = conv_outputs[0]

    heatmap = tf.reduce_sum(tf.multiply(pooled_grads, conv_outputs), axis=-1)
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)

    return heatmap


In [19]:
# Load an image
img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0) / 255.0

# Generate Grad-CAM
heatmap = get_gradcam(model, img_array, layer_name="conv5_block3_out")  # ResNet50 last conv layer


In [20]:
# Resize and colorize heatmap
heatmap = cv2.resize(heatmap, (224, 224))
heatmap = np.uint8(255 * heatmap)
heatmap_color = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

# Original image
orig = (img_array[0] * 255).astype("uint8")

# Blend heatmap with original image
overlay = cv2.addWeighted(orig, 0.6, heatmap_color, 0.4, 0)

plt.figure(figsize=(6,6))
plt.imshow(overlay)
plt.axis("off")
plt.title("Grad-CAM Visualization")
plt.savefig("../visualizations/gradcam_example.png")
plt.close()

print("Grad-CAM saved successfully!")


Grad-CAM saved successfully!
